# OpenSearch Warming Up (한글 형태소 분석기 사용 방법 이해)
>이 노트북은 SageMaker Studio* Data Science 3.0 kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.



여기서는 OpenSearch 가 설치된 것을 가정하고, 한글 형태소 분석기의 사용하는 법을 알려 드립니다.

---
## Ref: 
- [Amazon OpenSearch Service로 검색 구현하기](https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/blog-reindex)
- [OpenSearch Python Client](https://opensearch.org/docs/1.3/clients/python-high-level/)

# 1. 환경 세팅

In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils import print_ww

# 2. OpenSearch Client 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
### 아래 셀에 다음의 정보가 입력이 되어야 합니다.
```
opensearch_domain_endpoint = "<Type Domain Endpoint>"
http_auth = (rag_user_name, rag_user_password) # Master username, Master password

```


In [3]:
from utils.rag import create_aws_opensearch_client, check_if_index_exists, delete_index
from utils.rag import create_index, add_doc, search_document

In [4]:
aws_region = 'us-east-1'
opensearch_cluster_domain = 'https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com'

os.environ["OpenSearch_UserName"] = "<Type UserName>" 
os.environ["OpenSearch_UserPassword"] = "<Type Password>" 


rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]

# OpenSearch Dashboards URL = https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com/_dashboards
opensearch_domain_endpoint = "https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com"
http_auth = (rag_user_name, rag_user_password) # Master username, Master password



aws_client = create_aws_opensearch_client(
                                     aws_region,
                                     opensearch_domain_endpoint,
                                     http_auth)



# 3. 디폴트 Index Creation
- 간단하게 text 타입으로 title, body 두개의 컬럼으로 구성합니다.

In [5]:
index_name = 'default-index'
index_exists = check_if_index_exists(aws_client, index_name)

if index_exists:
    delete_index(aws_client, index_name)    
else:
    print("Index does not exist")    
    
# Create an index with non-default settings.

index_body = {
"mappings": {
    "properties": {
      "title": {
        "type": "text"
      },
      "body": { 
        "type": "text"
      }
    }
  }
}

create_index(aws_client, index_name, index_body)
index_info = aws_client.indices.get(index=index_name)
index_info

index_name=default-index, exists=False
Index does not exist

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'default-index'}


{'default-index': {'aliases': {},
  'mappings': {'properties': {'body': {'type': 'text'},
    'title': {'type': 'text'}}},
  'settings': {'index': {'creation_date': '1697368034480',
    'number_of_shards': '5',
    'number_of_replicas': '1',
    'uuid': 'cwD-H1QJS6KnhnTeRStMjg',
    'version': {'created': '136287827'},
    'provided_name': 'default-index'}}}}

# 4. 디폴트 Index 에 Doc 넣기
- 아래와 같이 문서 하나를 추가 합니다.

In [6]:

# index document
doc_body = {
  "title": "AWS Ground Station – 위성 데이터 수집 및 처리 서비스 정식 출시",
  "body": "지난 가을, AWS Ground Station에 대한 게시물에서 위성 데이터를 다운링크할 때 수행하는 단계를 미리 살펴봤습니다. 그동안 미리보기로 제공하던 AWS Ground Station을 정식 출시하고, 우선 미국 내 지상국 두 곳을 통해 바로 사용이 가능합니다. AWS Ground Station 사용하기 당시 설명한 것과 같이 첫 번째 단계는 위성의 NORAD ID 및 기타 정보를 AWS와 공유하여 AWS 계정에 위성을..."
}

add_doc(aws_client, index_name, doc_body, id='1')



Adding document:
{'_index': 'default-index', '_id': '1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


# 5. 문서 검색

## 'Title" 에 "출시" 단어를 검색합니다.

In [7]:
q = '출시'
query = {
  "query": {
    "match": {
      "title": {
        "query": f"{q}"
      }
    }
  }
}
response = search_document(aws_client, query, index_name)    
response


Search results:


{'took': 3,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.2876821,
  'hits': [{'_index': 'default-index',
    '_id': '1',
    '_score': 0.2876821,
    '_source': {'title': 'AWS Ground Station – 위성 데이터 수집 및 처리 서비스 정식 출시',
     'body': '지난 가을, AWS Ground Station에 대한 게시물에서 위성 데이터를 다운링크할 때 수행하는 단계를 미리 살펴봤습니다. 그동안 미리보기로 제공하던 AWS Ground Station을 정식 출시하고, 우선 미국 내 지상국 두 곳을 통해 바로 사용이 가능합니다. AWS Ground Station 사용하기 당시 설명한 것과 같이 첫 번째 단계는 위성의 NORAD ID 및 기타 정보를 AWS와 공유하여 AWS 계정에 위성을...'}}]}}

## "body" 에 "출시" 단어를 검색 합니다.
- doc 1의 body에 AWS Ground Station을 정식 출시하고... 이라는 내용이 있지만 검색되지 않습니다.
- 참조: [형태소 분석기 사용하기](https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/stemming)

In [8]:
q = '출시'
query = {
  "query": {
    "match": {
      "body": {
        "query": f"{q}"
      }
    }
  }
}
response = search_document(aws_client, query, index_name)    
response


Search results:


{'took': 3,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

이유는 body의 termvector를 보면 알 수 있는데 출시하고는 term으로 저장되었으나 우리가 원하는 출시에 대해서는 저장되어 있지 않기 때문입니다. 아래의 termvectors Query를 사용해 현재 색인된 문서의 term vector를 확인 할 수 있습니다. Response에서 "출시하고"만 저장된 것을 확인하십시요.

## term_vector 확인
- 아래의 결과의 하단에 보면 아래와 같이 "출시하고" 가 하나의 term 으로 저장이 된 것을 볼 수 있습니다.
```
'출시하고': {'term_freq': 1,
     'tokens': [{'position': 22, 'start_offset': 110, 'end_offset': 114}]},
```

In [9]:
aws_client.termvectors(index=index_name, id='1', fields='body')

{'_index': 'default-index',
 '_id': '1',
 '_version': 1,
 'found': True,
 'took': 0,
 'term_vectors': {'body': {'field_statistics': {'sum_doc_freq': 50,
    'doc_count': 1,
    'sum_ttf': 55},
   'terms': {'aws': {'term_freq': 4,
     'tokens': [{'position': 2, 'start_offset': 7, 'end_offset': 10},
      {'position': 18, 'start_offset': 87, 'end_offset': 90},
      {'position': 33, 'start_offset': 150, 'end_offset': 153},
      {'position': 52, 'start_offset': 228, 'end_offset': 231}]},
    'aws와': {'term_freq': 1,
     'tokens': [{'position': 50, 'start_offset': 218, 'end_offset': 222}]},
    'ground': {'term_freq': 3,
     'tokens': [{'position': 3, 'start_offset': 11, 'end_offset': 17},
      {'position': 19, 'start_offset': 91, 'end_offset': 97},
      {'position': 34, 'start_offset': 154, 'end_offset': 160}]},
    'id': {'term_freq': 1,
     'tokens': [{'position': 46, 'start_offset': 206, 'end_offset': 208}]},
    'norad': {'term_freq': 1,
     'tokens': [{'position': 45, 'start_

# 6. 은전한닢 형태소 분석기를 포함한 New Index 생성 및 실험

In [10]:
new_index_name = 'korea-index'
index_exists = check_if_index_exists(aws_client, new_index_name)

if index_exists:
    delete_index(aws_client, new_index_name)    
else:
    print("Index does not exist")    
        
# Create an index with non-default settings.

index_body = {
  "settings": {
    "analysis": {
      "tokenizer": {
        "seunjeon": {
          "type": "seunjeon_tokenizer"
        }
      },
      "analyzer": {
        "my_analyzer": {
          "type": "custom",
          "tokenizer": "seunjeon"
        }
      }
    }
  },
    "mappings": {
      "properties": {
        "title": {
          "type": "text"
        },
        "body": {
          "type": "text",
          "analyzer": "my_analyzer",
          "search_analyzer": "my_analyzer"
        }
      }
    }
  }

create_index(aws_client, new_index_name, index_body)
index_info = aws_client.indices.get(index=new_index_name)
index_info

index_name=korea-index, exists=False
Index does not exist

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'korea-index'}


{'korea-index': {'aliases': {},
  'mappings': {'properties': {'body': {'type': 'text',
     'analyzer': 'my_analyzer'},
    'title': {'type': 'text'}}},
  'settings': {'index': {'number_of_shards': '5',
    'provided_name': 'korea-index',
    'creation_date': '1697368094750',
    'analysis': {'analyzer': {'my_analyzer': {'type': 'custom',
       'tokenizer': 'seunjeon'}},
     'tokenizer': {'seunjeon': {'type': 'seunjeon_tokenizer'}}},
    'number_of_replicas': '1',
    'uuid': 'xtbnhNEmQdmVYpcp-qyUEA',
    'version': {'created': '136287827'}}}}}

## doc1 추가

In [11]:

# index document
doc_body = {
  "title": "AWS Ground Station – 위성 데이터 수집 및 처리 서비스 정식 출시",
  "body": "지난 가을, AWS Ground Station에 대한 게시물에서 위성 데이터를 다운링크할 때 수행하는 단계를 미리 살펴봤습니다. 그동안 미리보기로 제공하던 AWS Ground Station을 정식 출시하고, 우선 미국 내 지상국 두 곳을 통해 바로 사용이 가능합니다. AWS Ground Station 사용하기 당시 설명한 것과 같이 첫 번째 단계는 위성의 NORAD ID 및 기타 정보를 AWS와 공유하여 AWS 계정에 위성을..."
}

add_doc(aws_client, new_index_name, doc_body, id='1')



Adding document:
{'_index': 'korea-index', '_id': '1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


## body 에 "출시" 검색
- 아래와 같이 '출시" 를 할 경우에 검색이 되는 것을 확인할 수 있습니다.

In [12]:
q = '출시'
query = {
  "query": {
    "match": {
      "body": {
        "query": f"{q}"
      }
    }
  }
}
response = search_document(aws_client, query, new_index_name)    
response


Search results:


{'took': 40,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.32703072,
  'hits': [{'_index': 'korea-index',
    '_id': '1',
    '_score': 0.32703072,
    '_source': {'title': 'AWS Ground Station – 위성 데이터 수집 및 처리 서비스 정식 출시',
     'body': '지난 가을, AWS Ground Station에 대한 게시물에서 위성 데이터를 다운링크할 때 수행하는 단계를 미리 살펴봤습니다. 그동안 미리보기로 제공하던 AWS Ground Station을 정식 출시하고, 우선 미국 내 지상국 두 곳을 통해 바로 사용이 가능합니다. AWS Ground Station 사용하기 당시 설명한 것과 같이 첫 번째 단계는 위성의 NORAD ID 및 기타 정보를 AWS와 공유하여 AWS 계정에 위성을...'}}]}}

## term_vector 확인
- 아래와 같이 "출시하고" --> "출시" (명사), "출시하고" (EOJ) 두개로 형태소 분석을 하여 term_vectors 에 추가 되어 있는 것을 확인할 수 있습니다.
```
    '출시/N': {'term_freq': 1,
     'tokens': [{'position': 26, 'start_offset': 110, 'end_offset': 112}]},
    '출시하고/EOJ': {'term_freq': 1,
     'tokens': [{'position': 26, 'start_offset': 110, 'end_offset': 114}]},
```

In [13]:
aws_client.termvectors(index=new_index_name, id='1', fields='body')
# aws_client.termvectors(index=new_index_name, fields='body')

{'_index': 'korea-index',
 '_id': '1',
 '_version': 1,
 'found': True,
 'took': 19,
 'term_vectors': {'body': {'field_statistics': {'sum_doc_freq': 71,
    'doc_count': 1,
    'sum_ttf': 85},
   'terms': {'aws/SL': {'term_freq': 5,
     'tokens': [{'position': 2, 'start_offset': 7, 'end_offset': 10},
      {'position': 22, 'start_offset': 87, 'end_offset': 90},
      {'position': 38, 'start_offset': 150, 'end_offset': 153},
      {'position': 55, 'start_offset': 218, 'end_offset': 221},
      {'position': 57, 'start_offset': 228, 'end_offset': 231}]},
    'aws와/EOJ': {'term_freq': 1,
     'tokens': [{'position': 55, 'start_offset': 218, 'end_offset': 222}]},
    'ground/SL': {'term_freq': 3,
     'tokens': [{'position': 3, 'start_offset': 11, 'end_offset': 17},
      {'position': 23, 'start_offset': 91, 'end_offset': 97},
      {'position': 39, 'start_offset': 154, 'end_offset': 160}]},
    'id/SL': {'term_freq': 1,
     'tokens': [{'position': 51, 'start_offset': 206, 'end_offset': 20

# 7. 생성된 인덱스 삭제

In [14]:
index_exists = check_if_index_exists(aws_client, index_name)

if index_exists:
    delete_index(aws_client, index_name)    
else:
    print("Index does not exist")    

    
index_exists = check_if_index_exists(aws_client, new_index_name)

if index_exists:
    delete_index(aws_client, new_index_name)    
else:
    print("Index does not exist")    
    

index_name=default-index, exists=True

Deleting index:
{'acknowledged': True}
index_name=korea-index, exists=True

Deleting index:
{'acknowledged': True}
